### 강의에서 사용된 파이썬 주요 기능

### 경기 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', 40)

In [2]:
match_id = 2499719
match_events = pd.read_pickle(f'data/refined_events/England/{match_id}.pkl')
match_events

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
0,2499719,177959171,1H,2.759,1609,Arsenal,25413,A. Lacazette,Pass,Simple pass,[Accurate],50.96,34.68,32.24,14.96
1,2499719,177959172,1H,4.947,1609,Arsenal,370224,R. Holding,Pass,High pass,[Accurate],32.24,14.96,53.04,17.00
2,2499719,177959173,1H,6.542,1609,Arsenal,3319,M. Özil,Pass,Head pass,[Accurate],53.04,17.00,36.40,19.72
3,2499719,177959174,1H,8.143,1609,Arsenal,120339,Mohamed Elneny,Pass,Head pass,[Accurate],36.40,19.72,42.64,3.40
4,2499719,177959175,1H,10.302,1609,Arsenal,167145,Bellerín,Pass,Simple pass,[Accurate],42.64,3.40,74.88,8.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1775,2499719,177961040,2H,2992.492,1631,Leicester City,8480,K. Schmeichel,Pass,Launch,[Not accurate],7.28,31.96,47.84,34.68
1776,2499719,177961035,2H,2994.901,1609,Arsenal,49876,G. Xhaka,Pass,Head pass,[Accurate],56.16,33.32,75.92,28.56
1777,2499719,177961036,2H,2997.086,1609,Arsenal,7870,A. Ramsey,Shot,Shot,"[Right foot, Opportunity, Position: Out high r...",75.92,28.56,104.00,34.00
1778,2499719,177961041,2H,2998.964,1631,Leicester City,0,NaN,Interruption,Ball out of the field,[],3.12,35.36,NaN,NaN


### 경기 내 선수별 기록 집계

##### (1) 이벤트 유형을 기준으로 추출되는 기록의 집계

- 슈팅 횟수

In [3]:
shot_records = match_events[
    (match_events['event_type'] == 'Shot') |
    (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
]
shots = shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
shots.name = 'total_shots'
shots

team_id  team_name       player_id  player_name          
1609     Arsenal         3319       M. Özil                  3
                         7868       A. Oxlade-Chamberlain    6
                         7870       A. Ramsey                4
                         7945       D. Welbeck               3
                         14869      S. Kolašinac             1
                         25413      A. Lacazette             3
                         26010      O. Giroud                1
                         49876      G. Xhaka                 4
                         120339     Mohamed Elneny           1
                         167145     Bellerín                 1
1631     Leicester City  12829      J. Vardy                 2
                         14763      S. Okazaki               2
                         26150      R. Mahrez                3
Name: total_shots, dtype: int64

- 패스 횟수

In [4]:
pass_records = match_events[
    (match_events['event_type'] == 'Pass') |
    (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
]
passes = pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
passes.name = 'total_passes'
passes

team_id  team_name       player_id  player_name          
1609     Arsenal         3319       M. Özil                  82
                         3560       Nacho Monreal            58
                         7868       A. Oxlade-Chamberlain    55
                         7870       A. Ramsey                 9
                         7879       T. Walcott                2
                         7882       P. Čech                  14
                         7945       D. Welbeck               20
                         14869      S. Kolašinac             71
                         25413      A. Lacazette             20
                         26010      O. Giroud                 6
                         49876      G. Xhaka                 96
                         120339     Mohamed Elneny           68
                         167145     Bellerín                 56
                         370224     R. Holding               60
1631     Leicester City  8013       M. Albrigh

- 파울 횟수

In [5]:
foul_records = match_events[match_events['event_type'] == 'Foul']
fouls = foul_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
fouls.name = 'fouls'
fouls

team_id  team_name       player_id  player_name          
1609     Arsenal         7868       A. Oxlade-Chamberlain    1
                         7945       D. Welbeck               1
                         14869      S. Kolašinac             1
                         25413      A. Lacazette             2
                         49876      G. Xhaka                 1
                         167145     Bellerín                 1
                         370224     R. Holding               2
1631     Leicester City  8488       W. Morgan                4
                         8653       H. Maguire               1
                         14763      S. Okazaki               3
                         192748     M. James                 3
                         217078     D. Amartey               1
Name: fouls, dtype: int64

- 오프사이드 횟수

In [6]:
offside_records = match_events[match_events['event_type'] == 'Offside']
offsides = offside_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
offsides.name = 'offsides'
offsides

team_id  team_name       player_id  player_name 
1609     Arsenal         3319       M. Özil         2
                         7879       T. Walcott      1
                         7945       D. Welbeck      1
                         25413      A. Lacazette    2
1631     Leicester City  12829      J. Vardy        1
                         26150      R. Mahrez       1
Name: offsides, dtype: int64

##### (2) 태그 정보를 기준으로 추출되는 기록의 집계

- 유효슈팅 횟수

In [7]:
acc_shot_records = shot_records[shot_records['tags'].apply(lambda x: 'Accurate' in x)]
acc_shots = acc_shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
acc_shots.name = 'shots_on_target'
acc_shots

team_id  team_name       player_id  player_name          
1609     Arsenal         7868       A. Oxlade-Chamberlain    2
                         7870       A. Ramsey                1
                         7945       D. Welbeck               1
                         14869      S. Kolašinac             1
                         25413      A. Lacazette             2
                         26010      O. Giroud                1
                         49876      G. Xhaka                 1
                         167145     Bellerín                 1
1631     Leicester City  12829      J. Vardy                 2
                         14763      S. Okazaki               1
                         26150      R. Mahrez                1
Name: shots_on_target, dtype: int64

- 신체 부위별 슈팅 횟수

In [8]:
rshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Right foot' in x)]
rshots = rshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
rshots.name = 'rfoot_shots'

lshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Left foot' in x)]
lshots = lshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
lshots.name = 'lfoot_shots'

hshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Head/body' in x)]
hshots = hshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
hshots.name = 'header_shots'

shot_stats_list = [shots, acc_shots, rshots, lshots, hshots]
shot_stats = pd.concat(shot_stats_list, axis=1).fillna(0).astype(int)
shot_stats

total_shots  \
team_id team_name      player_id player_name                          
1609    Arsenal        3319      M. Özil                          3   
                       7868      A. Oxlade-Chamberlain            6   
                       7870      A. Ramsey                        4   
                       7945      D. Welbeck                       3   
                       14869     S. Kolašinac                     1   
                       25413     A. Lacazette                     3   
                       26010     O. Giroud                        1   
                       49876     G. Xhaka                         4   
                       120339    Mohamed Elneny                   1   
                       167145    Bellerín                         1   
1631    Leicester City 12829     J. Vardy                         2   
                       14763     S. Okazaki                       2   
                       26150     R. Mahrez                        3   

                                                        shots_on_target  \
team_id team_name      player_id player_name                              
1609    Arsenal        3319      M. Özil                              0   
                       7868      A. Oxlade-Chamberlain                2   
                       7870      A. Ramsey                            1   
                       7945      D. Welbeck                           1   
                       14869     S. Kolašinac                         1   
                       25413     A. Lacazette                         2   
                       26010     O. Giroud                            1   
                       49876     G. Xhaka                             1   
                       120339    Mohamed Elneny                       0   
                       167145    Bellerín                             1   
1631    Leicester City 12829     J. Vardy                             2   
                       14763     S. Okazaki                           1   
                       26150     R. Mahrez                            1   

                                                        rfoot_shots  \
team_id team_name      player_id player_name                          
1609    Arsenal        3319      M. Özil                          0   
                       7868      A. Oxlade-Chamberlain            5   
                       7870      A. Ramsey                        2   
                       7945      D. Welbeck                       2   
                       14869     S. Kolašinac                     0   
                       25413     A. Lacazette                     2   
                       26010     O. Giroud                        0   
                       49876     G. Xhaka                         0   
                       120339    Mohamed Elneny                   1   
                       167145    Bellerín                         1   
1631    Leicester City 12829     J. Vardy                         1   
                       14763     S. Okazaki                       0   
                       26150     R. Mahrez                        1   

                                                        lfoot_shots  \
team_id team_name      player_id player_name                          
1609    Arsenal        3319      M. Özil                          3   
                       7868      A. Oxlade-Chamberlain            1   
                       7870      A. Ramsey                        0   
                       7945      D. Welbeck                       1   
                       14869     S. Kolašinac                     1   
                       25413     A. Lacazette                     1   
                       26010     O. Giroud                        0   
                       49876     G. Xhaka                         4   
                       120339    Mohamed Elneny                   0   
                 

- 득점·도움·자책골 횟수

In [9]:
goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
goals = goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
goals.name = 'goals'

assist_records = match_events[match_events['tags'].apply(lambda x: 'Assist' in x)]
assists = assist_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
assists.name = 'assists'

own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
own_goals = own_goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
own_goals.name = 'own_goals'

goal_stats_list = [goals, assists, own_goals]
goal_stats = pd.concat(goal_stats_list, axis=1).fillna(0).astype(int)
goal_stats

goals  assists  own_goals
team_id team_name      player_id player_name                              
1609    Arsenal        7870      A. Ramsey           1        0          0
                       7945      D. Welbeck          1        0          0
                       25413     A. Lacazette        1        0          0
                       26010     O. Giroud           1        0          0
1631    Leicester City 12829     J. Vardy            2        0          0
                       14763     S. Okazaki          1        0          0
1609    Arsenal        14869     S. Kolašinac        0        1          0
                       49876     G. Xhaka            0        2          0
                       120339    Mohamed Elneny      0        1          0
1631    Leicester City 8013      M. Albrighton       0        1          0
                       8653      H. Maguire          0        1          0
                       26150     R. Mahrez           0        1          0

- 성공한 패스 횟수 및 패스 성공률

In [10]:
acc_pass_records = pass_records[pass_records['tags'].apply(lambda x: 'Accurate' in x)]
acc_passes = acc_pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
acc_passes.name = 'acc_passes'

pass_stats = pd.concat([passes, acc_passes], axis=1).fillna(0).astype(int)
pass_stats['pass_accuracy'] = (pass_stats['acc_passes'] / pass_stats['total_passes']).round(2)
pass_stats

total_passes  \
team_id team_name      player_id player_name                           
1609    Arsenal        3319      M. Özil                          82   
                       3560      Nacho Monreal                    58   
                       7868      A. Oxlade-Chamberlain            55   
                       7870      A. Ramsey                         9   
                       7879      T. Walcott                        2   
                       7882      P. Čech                          14   
                       7945      D. Welbeck                       20   
                       14869     S. Kolašinac                     71   
                       25413     A. Lacazette                     20   
                       26010     O. Giroud                         6   
                       49876     G. Xhaka                         96   
                       120339    Mohamed Elneny                   68   
                       167145    Bellerín                         56   
                       370224    R. Holding                       60   
1631    Leicester City 8013      M. Albrighton                    22   
                       8480      K. Schmeichel                    25   
                       8488      W. Morgan                        17   
                       8653      H. Maguire                       23   
                       12829     J. Vardy                          9   
                       14763     S. Okazaki                       15   
                       14853     C. Fuchs                         32   
                       26150     R. Mahrez                        28   
                       149019    D. Simpson                       27   
                       192748    M. James                         15   
                       217078    D. Amartey                        5   
                       265366    W. Ndidi                         22   
                       283142    D. Gray                           3   
                       285508    K. Iheanacho                      2   

                                                        acc_passes  \
team_id team_name      player_id player_name                         
1609    Arsenal        3319      M. Özil                        69   
                       3560      Nacho Monreal                  56   
                       7868      A. Oxlade-Chamberlain          40   
                       7870      A. Ramsey                       7   
                       7879      T. Walcott                      1   
                       7882      P. Čech                        13   
                       7945      D. Welbeck                     16   
                       14869     S. Kolašinac                   58   
                       25413     A. Lacazette                   15   
                       26010     O. Giroud                       6   
                       49876     G. Xhaka                       80   
                       120339    Mohamed Elneny                 63   
                       167145    Bellerín                       51   
                       370224    R. Holding                     48   
1631    Leicester City 8013      M. Albrighton                  14   
                       8480      K. Schmeichel                  18   
                       8488      W. Morgan                      16   
                       8653      H. Maguire                     22   
                       12829     J. Vardy                        7   
                       14763     S. Okazaki                      9   
                       14853     C. Fuchs                       20   
                       26150     R. Mahrez                      22   
                       149019    D. Simpson                     17   
                       192748    M. James                       12   
                       217078    D. Amartey                      4   
     

- 경고·퇴장 횟수

In [11]:
yellow_records = foul_records[foul_records['tags'].apply(lambda x: 'Yellow card' in x)]
yellows = yellow_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
yellows.name = 'yellow_cards'

red_records = foul_records[foul_records['tags'].apply(lambda x: 'Red card' in x)]
reds = red_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
reds.name = 'red_cards'

foul_stats = pd.concat([fouls, offsides, yellows, reds], axis=1).fillna(0).astype(int)
foul_stats

fouls  offsides  \
team_id team_name      player_id player_name                              
1609    Arsenal        7868      A. Oxlade-Chamberlain      1         0   
                       7945      D. Welbeck                 1         1   
                       14869     S. Kolašinac               1         0   
                       25413     A. Lacazette               2         2   
                       49876     G. Xhaka                   1         0   
                       167145    Bellerín                   1         0   
                       370224    R. Holding                 2         0   
1631    Leicester City 8488      W. Morgan                  4         0   
                       8653      H. Maguire                 1         0   
                       14763     S. Okazaki                 3         0   
                       192748    M. James                   3         0   
                       217078    D. Amartey                 1         0   
1609    Arsenal        3319      M. Özil                    0         2   
                       7879      T. Walcott                 0         1   
1631    Leicester City 12829     J. Vardy                   0         1   
                       26150     R. Mahrez                  0         1   

                                                        yellow_cards  \
team_id team_name      player_id player_name                           
1609    Arsenal        7868      A. Oxlade-Chamberlain             0   
                       7945      D. Welbeck                        0   
                       14869     S. Kolašinac                      0   
                       25413     A. Lacazette                      0   
                       49876     G. Xhaka                          0   
                       167145    Bellerín                          0   
                       370224    R. Holding                        0   
1631    Leicester City 8488      W. Morgan                         1   
                       8653      H. Maguire                        0   
                       14763     S. Okazaki                        0   
                       192748    M. James                          0   
                       217078    D. Amartey                        0   
1609    Arsenal        3319      M. Özil                           0   
                       7879      T. Walcott                        0   
1631    Leicester City 12829     J. Vardy                          0   
                       26150     R. Mahrez                         0   

                                                        red_cards  
team_id team_name      player_id player_name                       
1609    Arsenal        7868      A. Oxlade-Chamberlain          0  
                       7945      D. Welbeck                     0  
                       14869     S. Kolašinac                   0  
                       25413     A. Lacazette                   0  
                       49876     G. Xhaka                       0  
                       167145    Bellerín                       0  
                       370224    R. Holding                     0  
1631    Leicester City 8488      W. Morgan                      0  
                       8653      H. Maguire                     0  
                       14763     S. Okazaki                     0  
                       192748    M. James                       0  
                       217078    D. Amartey                     0  
1609    Arsenal        3319      M. Özil                        0  
                       7879      T. Walcott                     0  
1631    Leicester City 12829     J. Vardy                       0  
                       26150     R. Mahrez                      0

##### (3) 경기 내 선수별 기록 정리

In [12]:
player_stats = pd.concat([goal_stats, shot_stats, foul_stats, pass_stats], axis=1, sort=True)
player_stats

goals  assists  \
team_id team_name      player_id player_name                             
1609    Arsenal        3319      M. Özil                  NaN      NaN   
                       3560      Nacho Monreal            NaN      NaN   
                       7868      A. Oxlade-Chamberlain    NaN      NaN   
                       7870      A. Ramsey                1.0      0.0   
                       7879      T. Walcott               NaN      NaN   
                       7882      P. Čech                  NaN      NaN   
                       7945      D. Welbeck               1.0      0.0   
                       14869     S. Kolašinac             0.0      1.0   
                       25413     A. Lacazette             1.0      0.0   
                       26010     O. Giroud                1.0      0.0   
                       49876     G. Xhaka                 0.0      2.0   
                       120339    Mohamed Elneny           0.0      1.0   
                       167145    Bellerín                 NaN      NaN   
                       370224    R. Holding               NaN      NaN   
1631    Leicester City 8013      M. Albrighton            0.0      1.0   
                       8480      K. Schmeichel            NaN      NaN   
                       8488      W. Morgan                NaN      NaN   
                       8653      H. Maguire               0.0      1.0   
                       12829     J. Vardy                 2.0      0.0   
                       14763     S. Okazaki               1.0      0.0   
                       14853     C. Fuchs                 NaN      NaN   
                       26150     R. Mahrez                0.0      1.0   
                       149019    D. Simpson               NaN      NaN   
                       192748    M. James                 NaN      NaN   
                       217078    D. Amartey               NaN      NaN   
                       265366    W. Ndidi                 NaN      NaN   
                       283142    D. Gray                  NaN      NaN   
                       285508    K. Iheanacho             NaN      NaN   

                                                        own_goals  \
team_id team_name      player_id player_name                        
1609    Arsenal        3319      M. Özil                      NaN   
                       3560      Nacho Monreal                NaN   
                       7868      A. Oxlade-Chamberlain        NaN   
                       7870      A. Ramsey                    0.0   
                       7879      T. Walcott                   NaN   
                       7882      P. Čech                      NaN   
                       7945      D. Welbeck                   0.0   
                       14869     S. Kolašinac                 0.0   
                       25413     A. Lacazette                 0.0   
                       26010     O. Giroud                    0.0   
                       49876     G. Xhaka                     0.0   
                       120339    Mohamed Elneny               0.0   
                       167145    Bellerín                     NaN   
                       370224    R. Holding                   NaN   
1631    Leicester City 8013      M. Albrighton                0.0   
                       8480      K. Schmeichel                NaN   
                       8488      W. Morgan                    NaN   
                       8653      H. Maguire                   0.0   
                       12829     J. Vardy                     0.0   
                       14763     S. Okazaki                   0.0   
                       14853     C. Fuchs                     NaN   
                       26150     R. Mahrez                    0.0   
                       149019    D. Simpson                   NaN   
                       192748    M. James                     NaN   
                       217078    D. Amart

In [13]:
player_stats = player_stats.fillna(0).reset_index()

for col in player_stats.columns[4:]:
    if col != 'pass_accuracy':
        player_stats[col] = player_stats[col].astype(int)

player_stats

,team_id,team_name,player_id,player_name,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes,pass_accuracy
0,1609,Arsenal,3319,M. Özil,0,0,0,3,0,0,3,0,0,2,0,0,82,69,0.84
1,1609,Arsenal,3560,Nacho Monreal,0,0,0,0,0,0,0,0,0,0,0,0,58,56,0.97
2,1609,Arsenal,7868,A. Oxlade-Chamberlain,0,0,0,6,2,5,1,0,1,0,0,0,55,40,0.73
3,1609,Arsenal,7870,A. Ramsey,1,0,0,4,1,2,0,2,0,0,0,0,9,7,0.78
4,1609,Arsenal,7879,T. Walcott,0,0,0,0,0,0,0,0,0,1,0,0,2,1,0.50
5,1609,Arsenal,7882,P. Čech,0,0,0,0,0,0,0,0,0,0,0,0,14,13,0.93
6,1609,Arsenal,7945,D. Welbeck,1,0,0,3,1,2,1,0,1,1,0,0,20,16,0.80
7,1609,Arsenal,14869,S. Kolašinac,0,1,0,1,1,0,1,0,1,0,0,0,71,58,0.82
8,1609,Arsenal,25413,A. Lacazette,1,0,0,3,2,2,1,0,2,2,0,0,20,15,0.75
9,1609,Arsenal,26010,O. Giroud,1,0,0,1,1,0,0,1,0,0,0,0,6,6,1.00


### 경기 내 선수별 출전 시간 산출

##### (1) 선수교체 및 퇴장 기록 필터링

In [14]:
player_change_records = match_events[
    (match_events['event_type'] == 'Substitution') |
    (match_events['tags'].apply(lambda x: 'Red card' in x))
]
player_change_records

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
1290,2499719,0,2H,1260.0,1609,Arsenal,26010,O. Giroud,Substitution,Player in,[370224],NaN,NaN,NaN,NaN
1291,2499719,0,2H,1260.0,1609,Arsenal,370224,R. Holding,Substitution,Player out,[26010],NaN,NaN,NaN,NaN
1292,2499719,0,2H,1260.0,1609,Arsenal,7870,A. Ramsey,Substitution,Player in,[120339],NaN,NaN,NaN,NaN
1293,2499719,0,2H,1260.0,1609,Arsenal,120339,Mohamed Elneny,Substitution,Player out,[7870],NaN,NaN,NaN,NaN
1369,2499719,0,2H,1560.0,1631,Leicester City,217078,D. Amartey,Substitution,Player in,[14763],NaN,NaN,NaN,NaN
1370,2499719,0,2H,1560.0,1631,Leicester City,14763,S. Okazaki,Substitution,Player out,[217078],NaN,NaN,NaN,NaN
1440,2499719,0,2H,1740.0,1609,Arsenal,7879,T. Walcott,Substitution,Player in,[7945],NaN,NaN,NaN,NaN
1441,2499719,0,2H,1740.0,1609,Arsenal,7945,D. Welbeck,Substitution,Player out,[7879],NaN,NaN,NaN,NaN
1568,2499719,0,2H,2160.0,1631,Leicester City,285508,K. Iheanacho,Substitution,Player in,[192748],NaN,NaN,NaN,NaN
1569,2499719,0,2H,2160.0,1631,Leicester City,192748,M. James,Substitution,Player out,[285508],NaN,NaN,NaN,NaN


##### (2) 선발 선수 추출

In [15]:
in_players = player_change_records[player_change_records['sub_event_type'] == 'Player in']['player_id'].tolist()
player_ids = [p for p in match_events['player_id'].unique() if not p in in_players]
player_ids

[25413,
 370224,
 3319,
 120339,
 167145,
 8653,
 8013,
 0,
 49876,
 14869,
 3560,
 265366,
 14853,
 14763,
 12829,
 7882,
 192748,
 7945,
 149019,
 8488,
 7868,
 8480,
 26150]

##### (3) 선수교체 및 퇴장 기록을 활용한 phase 구분

In [16]:
period_durations = match_events.groupby('period')['time'].max()
phase_record_list = []
phase = 1

for period in period_durations.index:
    change_times = player_change_records[player_change_records['period'] == period]['time'].unique().tolist()
    change_times.append(period_durations[period])
    if 0 not in change_times:
        change_times = [0] + change_times

    for i in range(len(change_times[:-1])):
        moment_records = player_change_records[
            (player_change_records['period'] == period) &
            (player_change_records['time'] == change_times[i])
        ]

        for _, record in moment_records.iterrows():
            if record['sub_event_type'] == 'Player out' or record['event_type'] == 'Foul':
                player_ids.remove(record['player_id'])
            else:
                player_ids.append(record['player_id'])

        phase_record = {
            'phase': phase,
            'period': period,
            'start_time': change_times[i],
            'end_time': change_times[i+1],
            'duration': change_times[i+1] - change_times[i],
            'player_ids': player_ids.copy()
        }
        phase += 1

        phase_record_list.append(phase_record)

phase_records = pd.DataFrame(phase_record_list).set_index('phase')
phase_records

,period,start_time,end_time,duration,player_ids
phase,,,,,
1,1H,0.0,2852.557,2852.557,"[25413, 370224, 3319, 120339, 167145, 8653, 80..."
2,2H,0.0,1260.000,1260.000,"[25413, 370224, 3319, 120339, 167145, 8653, 80..."
3,2H,1260.0,1560.000,300.000,"[25413, 3319, 167145, 8653, 8013, 0, 49876, 14..."
4,2H,1560.0,1740.000,180.000,"[25413, 3319, 167145, 8653, 8013, 0, 49876, 14..."
5,2H,1740.0,2160.000,420.000,"[25413, 3319, 167145, 8653, 8013, 0, 49876, 14..."
6,2H,2160.0,2520.000,360.000,"[25413, 3319, 167145, 8653, 8013, 0, 49876, 14..."
7,2H,2520.0,3006.647,486.647,"[25413, 3319, 167145, 8653, 0, 49876, 14869, 3..."


##### (4) 선수별 각 phase 출전 여부 판단

In [17]:
player_ids = np.sort(match_events['player_id'].unique())
for player_id in player_ids:
    phase_records[player_id] = 0

for phase in phase_records.index:
    for player_id in phase_records.at[phase, 'player_ids']:
        phase_records.at[phase, player_id] = 1

phase_records = phase_records[np.concatenate([phase_records.columns[:4], player_ids])]
phase_records

,period,start_time,end_time,duration,0,3319,3560,7868,7870,7879,7882,7945,8013,8480,8488,8653,12829,14763,14853,14869,25413,26010,26150,49876,120339,149019,167145,192748,217078,265366,283142,285508,370224
phase,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1H,0.0,2852.557,2852.557,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,0,1
2,2H,0.0,1260.000,1260.000,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,0,1
3,2H,1260.0,1560.000,300.000,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,0,0
4,2H,1560.0,1740.000,180.000,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0
5,2H,1740.0,2160.000,420.000,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0
6,2H,2160.0,2520.000,360.000,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,1,1,0,1,0
7,2H,2520.0,3006.647,486.647,1,1,1,1,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,0


##### (5) 선수별 출전 시간 산출

In [18]:
playing_times = pd.Series(index=player_ids, dtype='float')
for player_id in player_ids:
    playing_times[player_id] = phase_records[phase_records[player_id] == 1]['duration'].sum().round(1)
playing_times

0         5859.2
3319      5859.2
3560      5859.2
7868      5859.2
7870      1746.6
7879      1266.6
7882      5859.2
7945      4592.6
8013      5372.6
8480      5859.2
8488      5859.2
8653      5859.2
12829     5859.2
14763     4412.6
14853     5859.2
14869     5859.2
25413     5859.2
26010     1746.6
26150     5859.2
49876     5859.2
120339    4112.6
149019    5859.2
167145    5859.2
192748    5012.6
217078    1446.6
265366    5859.2
283142     486.6
285508     846.6
370224    4112.6
dtype: float64

In [19]:
playing_times = playing_times.reset_index()
playing_times.columns = ['player_id', 'playing_time']
playing_times

,player_id,playing_time
0,0,5859.2
1,3319,5859.2
2,3560,5859.2
3,7868,5859.2
4,7870,1746.6
5,7879,1266.6
6,7882,5859.2
7,7945,4592.6
8,8013,5372.6
9,8480,5859.2


##### (6) 선수별 기록에 출전 시간 정보 추가

In [20]:
player_stats = pd.merge(player_stats, playing_times)
player_stats

,team_id,team_name,player_id,player_name,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes,pass_accuracy,playing_time
0,1609,Arsenal,3319,M. Özil,0,0,0,3,0,0,3,0,0,2,0,0,82,69,0.84,5859.2
1,1609,Arsenal,3560,Nacho Monreal,0,0,0,0,0,0,0,0,0,0,0,0,58,56,0.97,5859.2
2,1609,Arsenal,7868,A. Oxlade-Chamberlain,0,0,0,6,2,5,1,0,1,0,0,0,55,40,0.73,5859.2
3,1609,Arsenal,7870,A. Ramsey,1,0,0,4,1,2,0,2,0,0,0,0,9,7,0.78,1746.6
4,1609,Arsenal,7879,T. Walcott,0,0,0,0,0,0,0,0,0,1,0,0,2,1,0.50,1266.6
5,1609,Arsenal,7882,P. Čech,0,0,0,0,0,0,0,0,0,0,0,0,14,13,0.93,5859.2
6,1609,Arsenal,7945,D. Welbeck,1,0,0,3,1,2,1,0,1,1,0,0,20,16,0.80,4592.6
7,1609,Arsenal,14869,S. Kolašinac,0,1,0,1,1,0,1,0,1,0,0,0,71,58,0.82,5859.2
8,1609,Arsenal,25413,A. Lacazette,1,0,0,3,2,2,1,0,2,2,0,0,20,15,0.75,5859.2
9,1609,Arsenal,26010,O. Giroud,1,0,0,1,1,0,0,1,0,0,0,0,6,6,1.00,1746.6


### 대회 전체 선수별 기록 집계

##### (1) 경기 내 선수별 기록 집계 함수 구현

In [21]:
def generate_player_stats(match_id):
    # Data loading
    match_events = pd.read_pickle(f'data/refined_events/England/{match_id}.pkl')
    match_events = match_events[match_events['period'] != 'P']

    # Goal stats
    goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
    goals = goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    goals.name = 'goals'

    own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
    own_goals = own_goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    own_goals.name = 'own_goals'

    assist_records = match_events[match_events['tags'].apply(lambda x: 'Assist' in x)]
    assists = assist_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    assists.name = 'assists'

    goal_stats_list = [goals, assists, own_goals]
    goal_stats = pd.concat(goal_stats_list, axis=1).fillna(0).astype(int)

    # Shot stats
    shot_records = match_events[
        (match_events['event_type'] == 'Shot') |
        (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
    ]
    shots = shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    shots.name = 'total_shots'

    acc_shot_records = shot_records[shot_records['tags'].apply(lambda x: 'Accurate' in x)]
    acc_shots = acc_shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    acc_shots.name = 'shots_on_target'

    rshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Right foot' in x)]
    rshots = rshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    rshots.name = 'rfoot_shots'

    lshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Left foot' in x)]
    lshots = lshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    lshots.name = 'lfoot_shots'

    hshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Head/body' in x)]
    hshots = hshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    hshots.name = 'header_shots'

    shot_stats_list = [shots, acc_shots, rshots, lshots, hshots]
    shot_stats = pd.concat(shot_stats_list, axis=1).fillna(0).astype(int)

    # Foul stats
    foul_records = match_events[match_events['event_type'] == 'Foul']
    fouls = foul_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    fouls.name = 'fouls'

    offside_records = match_events[match_events['event_type'] == 'Offside']
    offsides = offside_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    offsides.name = 'offsides'

    yellow_records = foul_records[foul_records['tags'].apply(lambda x: 'Yellow card' in x)]
    yellows = yellow_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    yellows.name = 'yellow_cards'

    red_records = foul_records[foul_records['tags'].apply(lambda x: 'Red card' in x)]
    reds = red_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    reds.name = 'red_cards'

    foul_stats = pd.concat([fouls, offsides, yellows, reds], axis=1).fillna(0).astype(int)

    # Pass stats
    pass_records = match_events[
        (match_events['event_type'] == 'Pass') |
        (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
    ]
    passes = pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    passes.name = 'total_passes'

    acc_pass_records = pass_records[pass_records['tags'].apply(lambda x: 'Accurate' in x)]
    acc_passes = acc_pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    acc_passes.name = 'acc_passes'

    pass_stats = pd.concat([passes, acc_passes], axis=1).fillna(0).astype(int)
    pass_stats['pass_accuracy'] = (pass_stats['acc_passes'] / pass_stats['total_passes']).round(2)

    # Playing time
    player_change_records = match_events[
        (match_events['event_type'] == 'Substitution') |
        (match_events['tags'].apply(lambda x: 'Red card' in x))
    ]
    in_players = player_change_records[player_change_records['sub_event_type'] == 'Player in']['player_id'].tolist()
    player_ids = [p for p in match_events['player_id'].unique() if not p in in_players]

    period_durations = match_events.groupby('period')['time'].max()
    phase_record_list = []
    phase = 1

    for period in period_durations.index:
        change_times = player_change_records[player_change_records['period'] == period]['time'].unique().tolist()
        change_times.append(period_durations[period])
        if 0 not in change_times:
            change_times = [0] + change_times

        for i in range(len(change_times[:-1])):
            moment_records = player_change_records[
                (player_change_records['period'] == period) &
                (player_change_records['time'] == change_times[i])
            ]

            for _, record in moment_records.iterrows():
                if record['sub_event_type'] == 'Player out' or record['event_type'] == 'Foul':
                    player_ids.remove(record['player_id'])
                else:
                    player_ids.append(record['player_id'])

            phase_record = {
                'phase': phase,
                'period': period,
                'start_time': change_times[i],
                'end_time': change_times[i+1],
                'duration': change_times[i+1] - change_times[i],
                'player_ids': player_ids.copy()
            }
            phase += 1

            phase_record_list.append(phase_record)

    phase_records = pd.DataFrame(phase_record_list).set_index('phase')

    player_ids = np.sort(match_events['player_id'].unique())
    for player_id in player_ids:
        phase_records[player_id] = 0

    for phase in phase_records.index:
        for player_id in phase_records.at[phase, 'player_ids']:
            phase_records.at[phase, player_id] = 1

    phase_records = phase_records[np.concatenate([phase_records.columns[:4], player_ids])]

    playing_times = pd.Series(index=player_ids, dtype='float')
    for player_id in player_ids:
        playing_times[player_id] = phase_records[phase_records[player_id] == 1]['duration'].sum().round(1)
    playing_times = playing_times.reset_index()
    playing_times.columns = ['player_id', 'playing_time']

    # Concatenation
    player_stats = pd.concat([goal_stats, shot_stats, foul_stats, pass_stats], axis=1, sort=True).fillna(0)
    for col in player_stats.columns:
        if col != 'pass_accuracy':
            player_stats[col] = player_stats[col].astype(int)

    player_stats = pd.merge(player_stats.reset_index(), playing_times)
    player_stats['match_id'] = match_id

    cols = player_stats.columns.tolist()
    cols = ['match_id'] + cols[:4] + ['playing_time'] + cols[4:-2]
    return player_stats[cols]

In [22]:
generate_player_stats(match_id)

,match_id,team_id,team_name,player_id,player_name,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes,pass_accuracy
0,2499719,1609,Arsenal,3319,M. Özil,5859.2,0,0,0,3,0,0,3,0,0,2,0,0,82,69,0.84
1,2499719,1609,Arsenal,3560,Nacho Monreal,5859.2,0,0,0,0,0,0,0,0,0,0,0,0,58,56,0.97
2,2499719,1609,Arsenal,7868,A. Oxlade-Chamberlain,5859.2,0,0,0,6,2,5,1,0,1,0,0,0,55,40,0.73
3,2499719,1609,Arsenal,7870,A. Ramsey,1746.6,1,0,0,4,1,2,0,2,0,0,0,0,9,7,0.78
4,2499719,1609,Arsenal,7879,T. Walcott,1266.6,0,0,0,0,0,0,0,0,0,1,0,0,2,1,0.50
5,2499719,1609,Arsenal,7882,P. Čech,5859.2,0,0,0,0,0,0,0,0,0,0,0,0,14,13,0.93
6,2499719,1609,Arsenal,7945,D. Welbeck,4592.6,1,0,0,3,1,2,1,0,1,1,0,0,20,16,0.80
7,2499719,1609,Arsenal,14869,S. Kolašinac,5859.2,0,1,0,1,1,0,1,0,1,0,0,0,71,58,0.82
8,2499719,1609,Arsenal,25413,A. Lacazette,5859.2,1,0,0,3,2,2,1,0,2,2,0,0,20,15,0.75
9,2499719,1609,Arsenal,26010,O. Giroud,1746.6,1,0,0,1,1,0,0,1,0,0,0,0,6,6,1.00


##### (2) 대회 전체 경기별 선수 기록 집계

- 경기 정보 불러오기

In [32]:
dataset_name = 'England'
match_df = pd.read_csv(f'data/refined_events/{dataset_name}/matches.csv', index_col=0, encoding='utf-8-sig')
match_df

,gameweek,datetime,venue,team1_id,team1_name,team1_goals,team2_id,team2_name,team2_goals,duration
match_id,,,,,,,,,,
2499719,1,2017-08-11 18:45:00,Emirates Stadium,1609,Arsenal,4,1631,Leicester City,3,Regular
2499727,1,2017-08-12 11:30:00,Vicarage Road Stadium,1644,Watford,3,1612,Liverpool,3,Regular
2499726,1,2017-08-12 14:00:00,St. Mary's Stadium,1619,Southampton,0,10531,Swansea City,0,Regular
2499721,1,2017-08-12 14:00:00,Stamford Bridge,1610,Chelsea,2,1646,Burnley,3,Regular
2499728,1,2017-08-12 14:00:00,The Hawthorns,1627,West Bromwich Albion,1,1659,AFC Bournemouth,0,Regular
...,...,...,...,...,...,...,...,...,...,...
2500092,38,2018-05-13 14:00:00,Anfield,1612,Liverpool,4,1651,Brighton & Hove Albion,0,Regular
2500091,38,2018-05-13 14:00:00,The John Smith's Stadium,1673,Huddersfield Town,0,1609,Arsenal,1,Regular
2500090,38,2018-05-13 14:00:00,Selhurst Park,1628,Crystal Palace,2,1627,West Bromwich Albion,0,Regular


- 경기별로 선수 기록 집계 함수 호출

In [24]:
stats_list = []

for match_id in tqdm(match_df.index):
    match_player_stats = generate_player_stats(match_id)
    stats_list.append(match_player_stats)

player_stats = pd.concat(stats_list, ignore_index=True)
player_stats

100%|██████████| 380/380 [00:19<00:00, 19.50it/s]


,match_id,team_id,team_name,player_id,player_name,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes,pass_accuracy
0,2499719,1609,Arsenal,3319,M. Özil,5859.2,0,0,0,3,0,0,3,0,0,2,0,0,82,69,0.84
1,2499719,1609,Arsenal,3560,Nacho Monreal,5859.2,0,0,0,0,0,0,0,0,0,0,0,0,58,56,0.97
2,2499719,1609,Arsenal,7868,A. Oxlade-Chamberlain,5859.2,0,0,0,6,2,5,1,0,1,0,0,0,55,40,0.73
3,2499719,1609,Arsenal,7870,A. Ramsey,1746.6,1,0,0,4,1,2,0,2,0,0,0,0,9,7,0.78
4,2499719,1609,Arsenal,7879,T. Walcott,1266.6,0,0,0,0,0,0,0,0,0,1,0,0,2,1,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10347,2500089,1659,AFC Bournemouth,62224,R. Fraser,5787.4,0,0,0,0,0,0,0,0,1,0,0,0,31,23,0.74
10348,2500089,1659,AFC Bournemouth,134102,N. Aké,5787.4,0,0,0,0,0,0,0,0,1,0,0,0,31,25,0.81
10349,2500089,1659,AFC Bournemouth,239411,T. Mings,3906.2,0,0,0,1,0,0,1,0,0,0,0,0,45,38,0.84
10350,2500089,1659,AFC Bournemouth,245813,L. Mousset,3546.2,0,0,0,2,1,2,0,0,1,0,0,0,23,15,0.65


##### (3) 대회 전체 선수 기록 누적 집계

In [25]:
grouped = player_stats.groupby(['team_id', 'team_name', 'player_id', 'player_name'])

player_stats_accum = grouped[player_stats.columns[5:-1]].sum()
player_stats_accum['pass_accuracy'] = (player_stats_accum['acc_passes'] / player_stats_accum['total_passes']).round(2)
player_stats_accum['matches'] = grouped['match_id'].count()

player_stats_accum = player_stats_accum[['matches'] + player_stats.columns[5:-1].tolist()].reset_index()
player_stats_accum

,team_id,team_name,player_id,player_name,matches,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes
0,1609,Arsenal,3319,M. Özil,26,137800.4,4,8,0,39,14,3,35,1,21,13,4,0,1820,1586
1,1609,Arsenal,3361,A. Sánchez,19,94860.3,7,3,0,69,28,65,2,2,24,14,4,0,854,633
2,1609,Arsenal,3560,Nacho Monreal,28,143373.3,5,2,0,21,8,5,13,3,21,2,3,0,1713,1547
3,1609,Arsenal,7855,L. Koscielny,25,141744.8,2,0,0,8,2,2,0,6,13,2,4,0,1696,1550
4,1609,Arsenal,7856,P. Mertesacker,6,23418.3,1,0,0,2,2,0,0,2,2,0,0,0,235,221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,10531,Swansea City,246866,A. Mawson,38,218184.8,2,1,0,12,4,5,0,7,21,1,3,0,1682,1433
522,10531,Swansea City,258162,Renato Sanches,12,44872.1,0,0,0,12,0,12,0,0,8,0,1,0,390,320
523,10531,Swansea City,288865,O. McBurnie,11,18842.1,0,1,0,3,1,3,0,0,3,0,0,0,80,64
524,10531,Swansea City,343951,T. Abraham,31,110956.7,5,1,0,39,16,25,6,8,16,8,0,0,241,178


##### (4) 분야별 Top 10 추출

- 득점 순위 Top 10

In [26]:
player_stats_accum.sort_values('goals', ascending=False, ignore_index=True)[:10]

,team_id,team_name,player_id,player_name,matches,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes
0,1612,Liverpool,120353,Mohamed Salah,36,183159.1,32,8,0,142,68,15,119,8,16,18,1,0,927,723
1,1624,Tottenham Hotspur,8717,H. Kane,37,195400.3,29,2,0,175,74,107,43,25,37,42,5,0,597,441
2,1625,Manchester City,8325,S. Agüero,25,125374.6,21,6,0,91,42,59,18,14,12,16,2,0,609,515
3,1631,Leicester City,12829,J. Vardy,37,205853.5,20,2,0,66,34,35,22,9,29,44,3,0,409,272
4,1625,Manchester City,11066,R. Sterling,33,164906.8,18,10,0,80,35,49,31,0,41,17,4,0,1175,1005
5,1611,Manchester United,7905,R. Lukaku,34,183371.8,16,7,0,80,41,14,52,14,31,18,4,0,618,446
6,1612,Liverpool,15808,Roberto Firmino,37,174541.7,15,4,0,80,37,47,20,13,57,14,1,0,1108,837
7,1609,Arsenal,25413,A. Lacazette,32,139209.0,14,4,0,65,36,48,16,1,29,22,1,0,657,520
8,1625,Manchester City,340386,Gabriel Jesus,29,106256.1,13,2,0,55,31,39,7,9,30,15,6,0,544,453
9,1651,Brighton & Hove Albion,8416,G. Murray,35,138865.5,12,0,0,50,24,30,8,12,61,35,9,0,530,354


- 득점 1위 해리 케인 경기별 기록

In [27]:
player_stats[player_stats['player_name'] == 'H. Kane']

,match_id,team_id,team_name,player_id,player_name,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes,pass_accuracy
237,2499725,1624,Tottenham Hotspur,8717,H. Kane,5775.0,0,0,0,6,1,2,2,2,1,0,1,0,14,14,1.00
511,2499737,1624,Tottenham Hotspur,8717,H. Kane,5836.8,0,0,0,7,3,5,0,2,3,0,1,0,17,10,0.59
799,2499746,1624,Tottenham Hotspur,8717,H. Kane,5760.2,0,0,0,10,3,6,2,2,0,0,0,0,20,12,0.60
925,2499752,1624,Tottenham Hotspur,8717,H. Kane,5170.9,2,0,0,5,3,3,2,0,3,0,0,0,21,14,0.67
1289,2499766,1624,Tottenham Hotspur,8717,H. Kane,5677.1,0,0,0,4,2,3,0,1,0,0,0,0,18,16,0.89
1372,2499778,1624,Tottenham Hotspur,8717,H. Kane,5410.1,2,0,0,6,2,3,2,1,2,5,0,0,24,13,0.54
1644,2499783,1624,Tottenham Hotspur,8717,H. Kane,5221.9,2,0,0,5,3,4,1,0,0,2,0,0,13,11,0.85
2058,2499797,1624,Tottenham Hotspur,8717,H. Kane,5106.3,0,0,0,7,4,2,5,0,1,2,0,0,14,12,0.86
2454,2499807,1624,Tottenham Hotspur,8717,H. Kane,5428.6,2,1,0,6,2,2,3,1,0,1,0,0,12,9,0.75
2901,2499827,1624,Tottenham Hotspur,8717,H. Kane,4686.3,0,0,0,2,1,1,0,1,0,1,0,0,16,16,1.00


- 득점 순위 Top 10 - 동률시 출전 시간이 적을수록 우위

In [28]:
player_stats_accum.sort_values(['goals', 'playing_time'], ascending=[False, True], ignore_index=True)[:10]

,team_id,team_name,player_id,player_name,matches,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes
0,1612,Liverpool,120353,Mohamed Salah,36,183159.1,32,8,0,142,68,15,119,8,16,18,1,0,927,723
1,1624,Tottenham Hotspur,8717,H. Kane,37,195400.3,29,2,0,175,74,107,43,25,37,42,5,0,597,441
2,1625,Manchester City,8325,S. Agüero,25,125374.6,21,6,0,91,42,59,18,14,12,16,2,0,609,515
3,1631,Leicester City,12829,J. Vardy,37,205853.5,20,2,0,66,34,35,22,9,29,44,3,0,409,272
4,1625,Manchester City,11066,R. Sterling,33,164906.8,18,10,0,80,35,49,31,0,41,17,4,0,1175,1005
5,1611,Manchester United,7905,R. Lukaku,34,183371.8,16,7,0,80,41,14,52,14,31,18,4,0,618,446
6,1612,Liverpool,15808,Roberto Firmino,37,174541.7,15,4,0,80,37,47,20,13,57,14,1,0,1108,837
7,1609,Arsenal,25413,A. Lacazette,32,139209.0,14,4,0,65,36,48,16,1,29,22,1,0,657,520
8,1625,Manchester City,340386,Gabriel Jesus,29,106256.1,13,2,0,55,31,39,7,9,30,15,6,0,544,453
9,1651,Brighton & Hove Albion,8416,G. Murray,35,138865.5,12,0,0,50,24,30,8,12,61,35,9,0,530,354


- 패스 횟수 Top 10

In [29]:
player_stats_accum.sort_values('total_passes', ascending=False, ignore_index=True)[:10]

,team_id,team_name,player_id,player_name,matches,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes
0,1609,Arsenal,49876,G. Xhaka,38,206976.5,1,7,0,66,17,3,63,0,60,0,10,0,3034,2684
1,1625,Manchester City,70086,N. Otamendi,34,190848.3,4,0,1,31,9,18,2,11,29,0,9,0,2991,2805
2,1625,Manchester City,105339,Fernandinho,34,184890.1,5,3,0,48,20,38,4,6,42,0,7,0,2882,2648
3,1610,Chelsea,25553,Azpilicueta,37,212735.5,2,6,1,12,3,9,1,2,22,1,1,0,2752,2398
4,1625,Manchester City,38021,K. De Bruyne,37,195996.5,7,16,0,91,39,67,24,0,31,2,2,0,2739,2220
5,1611,Manchester United,70122,N. Matić,36,199462.0,1,1,0,18,3,0,18,0,36,1,6,0,2526,2249
6,1624,Tottenham Hotspur,48,J. Vertonghen,36,204257.5,0,0,0,23,3,4,15,4,20,1,4,0,2408,2148
7,1625,Manchester City,8317,David Silva,29,154916.5,9,10,0,53,20,5,46,2,24,4,5,0,2406,2152
8,1625,Manchester City,8277,K. Walker,32,181742.1,0,4,1,13,1,12,1,0,20,2,3,0,2341,2074
9,1624,Tottenham Hotspur,54,C. Eriksen,37,204367.1,11,9,0,97,40,69,27,1,14,1,0,0,2292,1860


- 90분당 패스 횟수 Top 10

In [30]:
valid_stats = player_stats_accum[player_stats_accum['playing_time'] >= 5400].copy()
valid_stats['passes_per_90min'] = (valid_stats['total_passes'] / valid_stats['playing_time'] * 5400).round(1)

cols = valid_stats.columns[:6].tolist() + ['passes_per_90min']
valid_stats.sort_values('passes_per_90min', ascending=False, ignore_index=True)[cols][:10]

,team_id,team_name,player_id,player_name,matches,playing_time,passes_per_90min
0,1625,Manchester City,8324,Y. Touré,9,15759.4,139.5
1,1625,Manchester City,241564,O. Zinchenko,8,33459.2,107.8
2,1625,Manchester City,70086,N. Otamendi,34,190848.3,84.6
3,1625,Manchester City,8307,V. Kompany,17,89207.7,84.6
4,1625,Manchester City,105339,Fernandinho,34,184890.1,84.2
5,1625,Manchester City,8317,David Silva,29,154916.5,83.9
6,1625,Manchester City,134502,A. Laporte,9,51001.6,83.0
7,1609,Arsenal,120339,Mohamed Elneny,13,54867.7,82.5
8,1609,Arsenal,49876,G. Xhaka,38,206976.5,79.2
9,1612,Liverpool,7964,J. Henderson,27,138140.0,79.2


In [35]:
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', 40)

def generate_player_stats(match_id):
    # Data loading
    match_events = pd.read_pickle(f'data/refined_events/England/{match_id}.pkl')
    match_events = match_events[match_events['period'] != 'P']

    # Goal stats
    goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
    goals = goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    goals.name = 'goals'

    own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
    own_goals = own_goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    own_goals.name = 'own_goals'

    assist_records = match_events[match_events['tags'].apply(lambda x: 'Assist' in x)]
    assists = assist_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    assists.name = 'assists'

    goal_stats_list = [goals, assists, own_goals]
    goal_stats = pd.concat(goal_stats_list, axis=1).fillna(0).astype(int)

    # Shot stats
    shot_records = match_events[
        (match_events['event_type'] == 'Shot') |
        (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
    ]
    shots = shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    shots.name = 'total_shots'

    acc_shot_records = shot_records[shot_records['tags'].apply(lambda x: 'Accurate' in x)]
    acc_shots = acc_shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    acc_shots.name = 'shots_on_target'

    rshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Right foot' in x)]
    rshots = rshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    rshots.name = 'rfoot_shots'

    lshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Left foot' in x)]
    lshots = lshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    lshots.name = 'lfoot_shots'

    hshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Head/body' in x)]
    hshots = hshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    hshots.name = 'header_shots'

    shot_stats_list = [shots, acc_shots, rshots, lshots, hshots]
    shot_stats = pd.concat(shot_stats_list, axis=1).fillna(0).astype(int)

    # Foul stats
    foul_records = match_events[match_events['event_type'] == 'Foul']
    fouls = foul_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    fouls.name = 'fouls'

    offside_records = match_events[match_events['event_type'] == 'Offside']
    offsides = offside_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    offsides.name = 'offsides'

    yellow_records = foul_records[foul_records['tags'].apply(lambda x: 'Yellow card' in x)]
    yellows = yellow_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    yellows.name = 'yellow_cards'

    red_records = foul_records[foul_records['tags'].apply(lambda x: 'Red card' in x)]
    reds = red_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    reds.name = 'red_cards'

    foul_stats = pd.concat([fouls, offsides, yellows, reds], axis=1).fillna(0).astype(int)

    # Pass stats
    pass_records = match_events[
        (match_events['event_type'] == 'Pass') |
        (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
    ]
    passes = pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    passes.name = 'total_passes'

    acc_pass_records = pass_records[pass_records['tags'].apply(lambda x: 'Accurate' in x)]
    acc_passes = acc_pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    acc_passes.name = 'acc_passes'

    pass_stats = pd.concat([passes, acc_passes], axis=1).fillna(0).astype(int)
    pass_stats['pass_accuracy'] = (pass_stats['acc_passes'] / pass_stats['total_passes']).round(2)

    # Playing time
    player_change_records = match_events[
        (match_events['event_type'] == 'Substitution') |
        (match_events['tags'].apply(lambda x: 'Red card' in x))
    ]
    in_players = player_change_records[player_change_records['sub_event_type'] == 'Player in']['player_id'].tolist()
    player_ids = [p for p in match_events['player_id'].unique() if not p in in_players]

    period_durations = match_events.groupby('period')['time'].max()
    phase_record_list = []
    phase = 1

    for period in period_durations.index:
        change_times = player_change_records[player_change_records['period'] == period]['time'].unique().tolist()
        change_times.append(period_durations[period])
        if 0 not in change_times:
            change_times = [0] + change_times

        for i in range(len(change_times[:-1])):
            moment_records = player_change_records[
                (player_change_records['period'] == period) &
                (player_change_records['time'] == change_times[i])
            ]

            for _, record in moment_records.iterrows():
                if record['sub_event_type'] == 'Player out' or record['event_type'] == 'Foul':
                    player_ids.remove(record['player_id'])
                else:
                    player_ids.append(record['player_id'])

            phase_record = {
                'phase': phase,
                'period': period,
                'start_time': change_times[i],
                'end_time': change_times[i+1],
                'duration': change_times[i+1] - change_times[i],
                'player_ids': player_ids.copy()
            }
            phase += 1

            phase_record_list.append(phase_record)

    phase_records = pd.DataFrame(phase_record_list).set_index('phase')

    player_ids = np.sort(match_events['player_id'].unique())
    for player_id in player_ids:
        phase_records[player_id] = 0

    for phase in phase_records.index:
        for player_id in phase_records.at[phase, 'player_ids']:
            phase_records.at[phase, player_id] = 1

    phase_records = phase_records[np.concatenate([phase_records.columns[:4], player_ids])]

    playing_times = pd.Series(index=player_ids, dtype='float')
    for player_id in player_ids:
        playing_times[player_id] = phase_records[phase_records[player_id] == 1]['duration'].sum().round(1)
    playing_times = playing_times.reset_index()
    playing_times.columns = ['player_id', 'playing_time']

    # Concatenation
    player_stats = pd.concat([goal_stats, shot_stats, foul_stats, pass_stats], axis=1, sort=True).fillna(0)
    for col in player_stats.columns:
        if col != 'pass_accuracy':
            player_stats[col] = player_stats[col].astype(int)

    player_stats = pd.merge(player_stats.reset_index(), playing_times)
    player_stats['match_id'] = match_id

    cols = player_stats.columns.tolist()
    cols = ['match_id'] + cols[:4] + ['playing_time'] + cols[4:-2]
    return player_stats[cols]


dataset_name = 'England'
match_df = pd.read_csv(f'data/refined_events/{dataset_name}/matches.csv', index_col=0, encoding='utf-8-sig')
match_df


stats_list = []

for match_id in tqdm(match_df.index):
    match_player_stats = generate_player_stats(match_id)
    stats_list.append(match_player_stats)

player_stats = pd.concat(stats_list, ignore_index=True)


grouped = player_stats.groupby(['team_id', 'team_name', 'player_id', 'player_name'])

player_stats_accum = grouped[player_stats.columns[5:-1]].sum()
player_stats_accum['pass_accuracy'] = (player_stats_accum['acc_passes'] / player_stats_accum['total_passes']).round(2)
player_stats_accum['matches'] = grouped['match_id'].count()

player_stats_accum = player_stats_accum[['matches'] + player_stats.columns[5:-1].tolist()].reset_index()


player_stats_accum.sort_values('goals', ascending=False, ignore_index=True)[:10]

100%|██████████| 380/380 [00:19<00:00, 19.30it/s]


,team_id,team_name,player_id,player_name,matches,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes
0,1612,Liverpool,120353,Mohamed Salah,36,183159.1,32,8,0,142,68,15,119,8,16,18,1,0,927,723
1,1624,Tottenham Hotspur,8717,H. Kane,37,195400.3,29,2,0,175,74,107,43,25,37,42,5,0,597,441
2,1625,Manchester City,8325,S. Agüero,25,125374.6,21,6,0,91,42,59,18,14,12,16,2,0,609,515
3,1631,Leicester City,12829,J. Vardy,37,205853.5,20,2,0,66,34,35,22,9,29,44,3,0,409,272
4,1625,Manchester City,11066,R. Sterling,33,164906.8,18,10,0,80,35,49,31,0,41,17,4,0,1175,1005
5,1611,Manchester United,7905,R. Lukaku,34,183371.8,16,7,0,80,41,14,52,14,31,18,4,0,618,446
6,1612,Liverpool,15808,Roberto Firmino,37,174541.7,15,4,0,80,37,47,20,13,57,14,1,0,1108,837
7,1609,Arsenal,25413,A. Lacazette,32,139209.0,14,4,0,65,36,48,16,1,29,22,1,0,657,520
8,1625,Manchester City,340386,Gabriel Jesus,29,106256.1,13,2,0,55,31,39,7,9,30,15,6,0,544,453
9,1651,Brighton & Hove Albion,8416,G. Murray,35,138865.5,12,0,0,50,24,30,8,12,61,35,9,0,530,354
